# 7-2: Implementation of LeNet
- LeNet: 사람의 손글씨를 0~9까지 식별하는 네트워크
- 이론에서 확인했던 LeNet의 표를 기준으로 코드 작성

## Code.7-2-1: LeNet with Model Sub-classing

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class LeNet(Model):
  def __init__(self):
    super(LeNet, self).__init__()

    self.conv1 = Conv2D(filters=6, kernel_size=5, padding='same',
                        activation='tanh')
    self.conv1_pool = AveragePooling2D(pool_size=2, strides=2)

    self.conv2 = Conv2D(filters=16, kernel_size=5, padding='valid',
                        activation='tanh')
    self.conv2_pool = AveragePooling2D(pool_size=2, strides=2)

    self.conv3 = Conv2D(filters=120, kernel_size=5, padding='valid',
                        activation='tanh')
    self.flatten = Flatten()

    self.dense1 = Dense(units=84, activation='tanh')
    self.dense2 = Dense(units=10, activation='softmax')
  
  def call(self, x):
    print(f"x: {x.shape}")

    x = self.conv1(x)
    print(f"x: {x.shape}")
    x = self.conv1_pool(x)
    print(f"x: {x.shape}")

    x = self.conv2(x)
    print(f"x: {x.shape}")
    x = self.conv2_pool(x)
    print(f"x: {x.shape}")

    x = self.conv3(x)
    print(f"x: {x.shape}")
    x = self.flatten(x)
    print(f"x: {x.shape}")

    x = self.dense1(x)
    print(f"x: {x.shape}")
    x = self.dense2(x)
    print(f"x: {x.shape}")

model=LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

x: (32, 28, 28, 1)
x: (32, 28, 28, 6)
x: (32, 14, 14, 6)
x: (32, 10, 10, 16)
x: (32, 5, 5, 16)
x: (32, 1, 1, 120)
x: (32, 120)
x: (32, 84)
x: (32, 10)


## Code.7-2-2: LeNet with Hybrid Method

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

class ConvLayer(Layer):
  def __init__(self, filters, padding, pool=True):
    super(ConvLayer, self).__init__()
    self.pool = pool

    self.conv = Conv2D(filters=6, kernel_size=5, padding=padding,
                        activation='tanh')

    if pool == True:
      self.conv_pool = AveragePooling2D(pool_size=2, strides=2)

  def call(self, x):
    x = self.conv(x)
    if self.pool == True:
      x = self.conv_pool(x)
    return x


class LeNet(Model):
  def __init__(self):
    super(LeNet, self).__init__()

    self.conv1 = ConvLayer(filters=6, padding='same')
    self.conv2 = ConvLayer(filters=16, padding='valid')
    self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)

    self.flatten = Flatten()

    self.dense1 = Dense(units=84, activation='tanh')
    self.dense2 = Dense(units=10, activation='softmax')
  
  def call(self, x):
    print(f"x: {x.shape}")

    x = self.conv1(x)
    print(f"x: {x.shape}")

    x = self.conv2(x)
    print(f"x: {x.shape}")

    x = self.conv3(x)
    print(f"x: {x.shape}")

    x = self.dense1(x)
    print(f"x: {x.shape}")
    x = self.dense2(x)
    print(f"x: {x.shape}")

model=LeNet()

x = tf.random.normal(shape=(32, 28, 28, 1))
predictions = model(x)

x: (32, 28, 28, 1)
x: (32, 14, 14, 6)
x: (32, 5, 5, 6)
x: (32, 1, 1, 6)
x: (32, 1, 1, 84)
x: (32, 1, 1, 10)


## Code.7-2-3: Forward Propagation of LeNet

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

# 임의의 데이터 셋을 가져올 수 있음
from tensorflow.keras.datasets import mnist

from tensorflow.keras.losses import SparseCategoricalCrossentropy

### LeNet Implementation ###
class ConvLayer(Layer):
  def __init__(self, filters, padding, pool=True):
    super(ConvLayer, self).__init__()
    self.pool = pool

    self.conv = Conv2D(filters=6, kernel_size=5, padding=padding,
                        activation='tanh')

    if pool == True:
      self.conv_pool = AveragePooling2D(pool_size=2, strides=2)

  def call(self, x):
    x = self.conv(x)
    if self.pool == True:
      x = self.conv_pool(x)
    return x


class LeNet(Model):
  def __init__(self):
    super(LeNet, self).__init__()

    self.conv1 = ConvLayer(filters=6, padding='same')
    self.conv2 = ConvLayer(filters=16, padding='valid')
    self.conv3 = ConvLayer(filters=120, padding='valid', pool=False)

    self.flatten = Flatten()

    self.dense1 = Dense(units=84, activation='tanh')
    self.dense2 = Dense(units=10, activation='softmax')
  
  def call(self, x):
    print(f"x: {x.shape}")

    x = self.conv1(x)
    print(f"x: {x.shape}")

    x = self.conv2(x)
    print(f"x: {x.shape}")

    x = self.conv3(x)
    print(f"x: {x.shape}")

    x = self.dense1(x)
    print(f"x: {x.shape}")
    x = self.dense2(x)
    print(f"x: {x.shape}")

    return x

### Dataset Preparation ###
(train_images, train_labels), _ = mnist.load_data()
# color channel 생성 
train_images = np.expand_dims(train_images, axis=3).astype(np.float32)
train_labels = train_labels.astype(np.int32)

# print(train_images.shape, train_images.dtype) 
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.batch(32)

### Forward Propagation ###
model = LeNet()
loss_object = SparseCategoricalCrossentropy()

for images, labels in train_ds:
  predictions = model(images)
  loss = loss_object(labels, predictions)

  print(loss)

  break

x: (32, 28, 28, 1)
x: (32, 14, 14, 6)
x: (32, 5, 5, 6)
x: (32, 1, 1, 6)
x: (32, 1, 1, 84)
x: (32, 1, 1, 10)
tf.Tensor(2.28972, shape=(), dtype=float32)


## Code.7-1-4: Implementation with Model and Layer Sub-classing

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Layer

class MyConv(Layer):
  def __init__(self, n_neuron):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, padding=padding,
                 activation=activation)
    self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
  
  def call(self, x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

class TestCNN(Model):
  def __init__(self):
    super(TestCNN, self).__init__()

    self.conv1 = MyConv(n_conv_neurons[0])
    self.conv2 = MyConv(n_conv_neurons[1])
    self.conv3 = MyConv(n_conv_neurons[2])
    self.flatten = Flatten()

    self.dense1 = Dense(units=n_dense_neurons[0], activation=activation)
    self.dense2 = Dense(units=n_dense_neurons[1], activation=activation)
    self.dense3 = Dense(units=n_dense_neurons[2], activation='softmax')
  
  def call(self, x):
    x = self.conv1(x)
    x = self.conv1_pool(x)

    x = self.conv2(x)
    x = self.conv2_pool(x)

    x = self.conv3(x)
    x = self.conv3_pool(x)

    x = self.flatten(x)

    x = self.dense1(x)
    x = self.dense2(x)
    x = self.dense3(x)
    return x


In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Layer

class MyConv(Layer):
  def __init__(self, n_neuron):
    super(MyConv, self).__init__()

    self.conv = Conv2D(filters=n_neuron, kernel_size=k_size, padding=padding,
                 activation=activation)
    self.conv_pool = MaxPooling2D(pool_size=pool_size, strides=pool_strides)
  
  def call(self, x):
    x = self.conv(x)
    x = self.conv_pool(x)
    return x

class TestCNN(Model):
  def __init__(self):
    super(TestCNN, self).__init__()
    self.fe = Sequential()

    self.fe.add(MyConv(n_conv_neurons[0]))
    self.fe.add(MyConv(n_conv_neurons[1]))
    self.fe.add(MyConv(n_conv_neurons[2]))
    self.fe.add(Flatten())

    self.classifier = Sequential()
    self.classifier.add(Dense(units=n_dense_neurons[0], activation=activation))
    self.classifier.add(Dense(units=n_dense_neurons[1], activation=activation))
    self.classifier.add(Dense(units=n_dense_neurons[2], activation='softmax'))
  
  def call(self, x):
    x = self.fe(x)
    x = self.classifier(x)

    return x